In [0]:
DEBUG = False
CONFIG_FILE = '/datascope/subaru/data/targeting/m31/m31_PFS_22_SSP/import/m31_PFS_22_SSP_099/ga-import_20250830213515.config'
OUTPUT_PATH = '/datascope/subaru/data/targeting/m31/m31_PFS_22_SSP/import/m31_PFS_22_SSP_099'

# Plot the cross-matched target list

In [0]:
import os, sys
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
plt.rc('font', size=6) #controls default text size

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', int(os.environ['PFS_TARGETING_DEBUGPORT'])))
    debug = True

# Imports

In [0]:
import pfs.utils
from pfs.ga.targeting.instrument import *
from notebooks import *

# Load the netflow config file

In [0]:
# Load the configuration
config = load_netflow_config(CONFIG_FILE)

In [0]:
config.targets['fluxstd'].mask

# Load the cross-matched target list

In [0]:
fn = os.path.expandvars(os.path.join(OUTPUT_PATH, 'netflow_targets.feather'))
targets = pd.read_feather(fn)

In [0]:
targets.columns

In [0]:
targets['key'].unique()

In [0]:
# Number of targets in each target list
# Target lists are processed in order, each target is considered from a catalog when
# it first appears among the targets. If a target in a further target list has a match
# in the main target list, it's not added again but rather, the missing columns are
# updated.

for key in targets['key'].unique():
    mask = (targets['key'] == key)
    print(key, mask.sum())

In [0]:
# Histogram of proper motion

for key in targets['key'].unique():
    mask = (targets['key'] == key) & ~(targets['pmra'].isna())

    f, ax = plt.subplots()

    hist, bins = np.histogram(targets[mask]['pmra'])
    ax.step(0.5 * (bins[1:] + bins[:-1]), hist, where='mid')

    hist, bins = np.histogram(targets[mask]['pmdec'])
    ax.step(0.5 * (bins[1:] + bins[:-1]), hist, where='mid')

    ax.set_title(key)

In [0]:
# Fraction of stars with proper motion

keys = targets['key'].unique()
hist = np.zeros(keys.size, dtype=float)
bins = np.arange(keys.size)
for i, key in enumerate(keys):
    mask = (targets['key'] == key) & ~(targets['pmra'].isna())
    hist[i] = mask.sum() / mask.size

f, ax = plt.subplots(dpi=240)

ax.bar(bins, hist)
ax.set_xticks(bins)
ax.set_xticklabels(keys)

# Plot the target list

In [0]:
if config.field.center is not None:
    center = config.field.center.get_pointing()
else:
    center = config.pointings[0].get_pointing()

if config.pointings is not None:
    pointings = [ p.get_pointing() for p in config.pointings ]
else:
    pointings = None

pointings, center

In [0]:
# Create the PFI instrument. This will load the instrument calibration data.
pfi = SubaruPFI(instrument_options=config.instrument_options)

In [0]:
f, ax = plt.subplots()